# GAN2 (generate numbers what i induce) 

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('./mnist/data/',one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
total_epoch=100
batch_size=100
n_hidden=256
n_input=28*28
n_noise=128
n_class=10

## Placeholders

In [3]:
X=tf.placeholder(tf.float32,[None,n_input])
Y=tf.placeholder(tf.float32,[None,n_class])
Z=tf.placeholder(tf.float32,[None,n_noise])

## Generator

In [4]:
def generator(noise,labels):
    with tf.variable_scope('generator'):
        inputs=tf.concat([noise,labels],1)
        hidden=tf.layers.dense(inputs,n_hidden,activation=tf.nn.relu)
        output=tf.layers.dense(hidden,n_input,activation=tf.nn.sigmoid)
        
    return output

## Discriminator

In [5]:
def discriminator(inputs,labels,reuse=None):
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()
            
        inputs=tf.concat([inputs,labels],1)
        
        hidden=tf.layers.dense(inputs,n_hidden,activation=tf.nn.relu)
        output=tf.layers.dense(hidden,1,activation=None)
        
    return output

In [6]:
def get_noise(batch_size,n_noise):
    return np.random.uniform(-1.,1.,size=[batch_size,n_noise])

In [7]:
G=generator(Z,Y)
D_real=discriminator(X,Y)
D_gene=discriminator(G,Y,True)

## Loss function & Optimizer

In [8]:
loss_D_real=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real,labels=tf.ones_like(D_real)))
loss_D_gene=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene,labels=tf.zeros_like(D_gene)))
loss_D=loss_D_real+loss_D_gene ## minimization this

loss_G=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene,labels=tf.ones_like(D_gene)))

In [9]:
vars_D=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='discriminator')
vars_G=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='generator')
train_D=tf.train.AdamOptimizer().minimize(loss_D,var_list=vars_D)
train_G=tf.train.AdamOptimizer().minimize(loss_G,var_list=vars_G)

## Session

In [10]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
total_batch=int(mnist.train.num_examples/batch_size)
loss_val_D,loss_val_G=0,0

for epoch in range(total_epoch):
    for jmi in range(total_batch):
        batch_xs,batch_ys=mnist.train.next_batch(batch_size)
        noise=get_noise(batch_size,n_noise)
        
        _,loss_val_D=sess.run([train_D,loss_D],feed_dict={X:batch_xs,Y:batch_ys,Z:noise})
        _,loss_val_G=sess.run([train_G,loss_G],feed_dict={Y:batch_ys,Z:noise})
        
    print('Epoch:','%04d'%(epoch+1),'D loss: {:.4f}'.format(loss_val_D),'G loss: {:.4f}'.format(loss_val_G))
        
    if epoch==0 or (epoch+1)%10==0:
        sample_size=10
        noise=get_noise(sample_size,n_noise)
        samples=sess.run(G,feed_dict={Y:mnist.test.labels[:sample_size],Z:noise})

        fig,ax=plt.subplots(2,10,figsize=(sample_size,2))
        
        for jmi in range(sample_size):
            ax[0][jmi].set_axis_off()
            ax[1][jmi].set_axis_off()
            
            ax[0][jmi].imshow(np.reshape(mnist.test.images[jmi],(28,28)))
            ax[1][jmi].imshow(np.reshape(samples[jmi],(28,28)))
            
        plt.savefig('samples2/{}.png'.format(str(epoch).zfill(3)),bbox_inches='tight')
        
print('Optimization has been done!')
            

Epoch: 0001 D loss: 0.0051 G loss: 8.9146
Epoch: 0002 D loss: 0.0110 G loss: 8.5142
Epoch: 0003 D loss: 0.0032 G loss: 10.0568
Epoch: 0004 D loss: 0.0136 G loss: 7.4542
Epoch: 0005 D loss: 0.0271 G loss: 6.7865
Epoch: 0006 D loss: 0.0395 G loss: 6.6287
Epoch: 0007 D loss: 0.0592 G loss: 6.4998
Epoch: 0008 D loss: 0.0199 G loss: 6.9851
Epoch: 0009 D loss: 0.0883 G loss: 6.3330
Epoch: 0010 D loss: 0.0892 G loss: 6.3971
Epoch: 0011 D loss: 0.2505 G loss: 5.6947
Epoch: 0012 D loss: 0.1847 G loss: 4.7563
Epoch: 0013 D loss: 0.5095 G loss: 3.9340
Epoch: 0014 D loss: 0.2278 G loss: 4.2248
Epoch: 0015 D loss: 0.2679 G loss: 4.8708
Epoch: 0016 D loss: 0.3890 G loss: 3.8491
Epoch: 0017 D loss: 0.3961 G loss: 3.9635
Epoch: 0018 D loss: 0.5029 G loss: 4.1040
Epoch: 0019 D loss: 0.3511 G loss: 4.0852
Epoch: 0020 D loss: 0.7132 G loss: 3.6390
Epoch: 0021 D loss: 0.5525 G loss: 3.7963
Epoch: 0022 D loss: 0.6633 G loss: 2.9266
Epoch: 0023 D loss: 0.7792 G loss: 2.3561
Epoch: 0024 D loss: 0.7775 G loss